In [1]:
import os
import pandas as pd
import cv2
from detector import get_info
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

Using cache found in C:\Users\virgi/.cache\torch\hub\ultralytics_yolov5_master
requirements: pandas>=1.1.4 not found and is required by YOLOv5, attempting auto-update...
requirements: Command 'pip install 'pandas>=1.1.4' ' returned non-zero exit status 1.
YOLOv5  2022-5-16 Python-3.7.8 torch-1.8.1+cu102 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [2]:
model_folder = 'results/models/model_big_with_hand'
model = tf.keras.models.load_model(model_folder)

In [3]:
df_users = pd.read_csv('users.csv')
df_users.head()

,name_folder,user_id,sex,train
0,adrian,user01,0,1
1,alexis,user02,0,1
2,bea,user03,1,1
3,camino,user04,1,1
4,claudia,user05,1,1


In [4]:
for index, row in df_users.iterrows():
    name_folder = row.name_folder
    user_id = row.user_id
    user_identifier = int(user_id[4:])
    sex = row.sex
    
    print(user_id, sex)
        
    #List videos from user
    videos = [v for v in os.listdir(os.path.join('..','videos',name_folder)) if v.endswith('.avi') and 'sos.' in v]
    
    for video in videos:
        
        info_video = video.split('_')
        info_hand = info_video[1]
        hand =  'Right' if info_hand[0]== 'r' else 'Left'
        hand_id =  0 if info_hand[0]== 'r' else 1
        distance =  'Near' if info_hand[1]== 'n' else 'Far'
        print(video, hand)
        landmarks = np.array([])
        #Read and save every frame
        vidcap = cv2.VideoCapture(os.path.join('..','videos',name_folder,video))
        success,image = vidcap.read()
        frame = 1
        while success:
            image = cv2.flip(image, 1) # Mirror image
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            users = get_info(image)
            if len(users)==1 and users[0][hand] is not None:
                hand_points = users[0][hand]
                info = np.append([frame],hand_points)
                info = np.append(info, [hand_id])
                if landmarks.shape[0] == 0:
                    landmarks = np.array([info])
                else:
                    landmarks = np.append(landmarks,[info], axis=0)
            # Read new image
            frame +=1
            success,image = vidcap.read()
        if landmarks.shape[0]>0:
            predictions = model.predict(landmarks[:,1:], verbose=False)
            y_classes = predictions.argmax(axis=-1) + 1
            prob_prediction = predictions.max(axis=-1)
            #print(y_classes)
            data = np.append([landmarks[:,0]], [y_classes],axis=0).astype(np.int32)
            data = np.append(data, [prob_prediction], axis=0)
            df_video = pd.DataFrame(np.transpose(data), columns=['frame', 'sign', 'prob'])
            df_video.to_csv('sos_sequences/'+user_id+'_'+str(sex)+'_'+info_hand[:2]+'.csv', index=False)

user01 0
signo_lnsos.avi Left
signo_lfsos.avi Left
signo_rnsos.avi Right
signo_rfsos.avi Right
user02 0
signo_lfsos.avi Left
signo_rnsos.avi Right
signo_rfsos.avi Right
signo_lnsos.avi Left
user03 1
signo_rfsos.avi Right
signo_lnsos.avi Left
signo_lfsos.avi Left
signo_rnsos.avi Right
user04 1
signo_lnsos.avi Left
signo_lfsos.avi Left
signo_rnsos.avi Right
signo_rfsos.avi Right
user05 1
signo_lnsos.avi Left
signo_lfsos.avi Left
signo_rnsos.avi Right
signo_rfsos.avi Right
user06 0
signo_lnsos.avi Left
signo_lfsos.avi Left
signo_rnsos.avi Right
signo_rfsos.avi Right
user07 0
signo_lfsos.avi Left
signo_rnsos.avi Right
signo_rfsos.avi Right
signo_lnsos.avi Left
user08 1
signo_rnsos.avi Right
signo_rfsos.avi Right
signo_lfsos.avi Left
signo_lnsos.avi Left
user09 0
signo_rfsos.avi Right
signo_lnsos.avi Left
signo_lfsos.avi Left
signo_rnsos.avi Right
user10 1
signo_rnsos.avi Right
signo_rfsos.avi Right
signo_lnsos.avi Left
signo_lfsos.avi Left
user11 0
signo_rnsos.avi Right
signo_lfsos.avi Lef